# Neural Networks from Scratch

Following a YouTube tutorial

## Hardcoding inputs, weights and biases

In [4]:
inputs = [1, 2, 3, 2.5]
weights = [[0.2, 0.8, -0.5, 1.0],
          [0.5, -0.91, 0.26, -0.5],
          [-0.26, -0.27, 0.17, 0.87]]
biases = [2, 3, 0.5]

## Testing out zip() and for loops

In [5]:
a = zip(weights, biases)
for neuron_weights, neuron_bias in a:
    print(neuron_weights, neuron_bias)

[0.2, 0.8, -0.5, 1.0] 2
[0.5, -0.91, 0.26, -0.5] 3
[-0.26, -0.27, 0.17, 0.87] 0.5


## Hardcoding dot product of weights and biases

In [6]:
layer_outputs = [] # Output of the current layer
for neuron_weights, neuron_bias in zip(weights, biases):
    neuron_output = 0 # Output of a given neuron
    for n_input, weight in zip(inputs, neuron_weights):
        neuron_output += n_input*weight
    neuron_output += neuron_bias
    layer_outputs.append(neuron_output)

print(layer_outputs)

[4.8, 1.21, 2.385]


## Looking at numpy arrays and matrices

In [7]:
import numpy as np
l = np.array([1,5,6,2])
lol = np.array([[1,5,6,2],
              [3,2,1,3]])
print(l.shape, lol.shape)

(4,) (2, 4)


## Checking out numpy.dot

In [8]:
a = [1,2,3]
b = [2,3,4]

dot_product = a[0]*b[0] + a[1]*b[1] + a[2]*b[2]
print(dot_product)
print(np.dot(a, b))

20
20


## Analyzing shape for dot product

In [9]:
outputs = np.dot(weights, inputs) + biases
# wrong_order = np.dot(inputs, weights) + biases
print(outputs)
# print(wrong_order)

[4.8   1.21  2.385]


## Using a batch of inputs instead of a single set of inputs

In [10]:
input_batch = [[1,2,3,2.5],
               [2.0,5.0, -1.0, 2-0],
               [-1.5, 2.7, 3.3, -0.8]]
# dot product doesn't work anymore, because input shape changed
# outputs = np.dot(weights, input_batch) + biases
outputs = np.dot(input_batch, np.transpose(weights)) + biases
print(outputs)

[[ 4.8    1.21   2.385]
 [ 8.9   -1.81   0.2  ]
 [ 1.41   1.051  0.026]]


## Adding a second layer by hand and passing data through

In [11]:
weights2 = [[0.1, -0.14, 0.5],
            [-0.5, 0.12, -0.33],
            [-0.44, 0.73, -0.13]]
biases2 = [-1, 2, -0.5]

layer1_outputs = np.dot(input_batch, np.transpose(weights)) + biases
layer2_outputs = np.dot(layer1_outputs, np.transpose(weights2)) + biases2

print(layer2_outputs)

[[ 0.5031  -1.04185 -2.03875]
 [ 0.2434  -2.7332  -5.7633 ]
 [-0.99314  1.41254 -0.35655]]


## Defining a DenseLayer class with forward method

In [29]:
X = [[1, 2, 3, 2.5],
      [2.0, 5.0, -1.0, 2-0],
      [-1.5, 2.7, 3.3, -0.8]]

np.random.seed(0)

class DenseLayer:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.1 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((n_neurons))
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases
        
# print(0.1*np.random.randn(4, 3))

layer1 = DenseLayer(4, 5)
layer2 = DenseLayer(5, 2)

layer1.forward(X)
# print(layer1.output)
layer2.forward(layer1.output)
print(layer2.output)

[[ 0.148296   -0.08397602]
 [ 0.14100315 -0.01340469]
 [ 0.20124979 -0.07290616]]


## Defining ReLU

In [13]:
inputs = [0, 2, -1, 3.3, -2.7, 1.1, 2.2, -100]
output = []

for i in inputs:
    output.append(max(0, i))
        
print(output)

[0, 2, 0, 3.3, 0, 1.1, 2.2, 0]


In [14]:
class ReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

In [15]:
def createData(points, classes):
    X = np.zeros((points*classes, 2))
    y = np.zeros(points*classes, dtype='uint8')
    for class_number in range(classes):
        ix = range(points*class_number, points*(class_number+1))
        r = np.linspace(0.0, 1, points)
        t = np.linspace(class_number*4, (class_number+1)*4, points) + np.random.randn(points)*0.2
        X[ix] = np.c_[r*np.sin(t*2.5), r*np.cos(t*2.5)]
        y[ix] = class_number
    return X, y

In [16]:
import matplotlib.pyplot as plt

def plotData(X, y):
    plt.scatter(X[:,0], X[:,1])
    plt.show()

    plt.scatter(X[:,0], X[:,1], c=y, cmap="brg")
    plt.show()

In [17]:
X, y = createData(100, 3)
# plotData(X, y)

layer1 = DenseLayer(2, 5)
activation1 = ReLU()

layer1.forward(X)

activation1.forward(layer1.output)
print(activation1.output)

[[0.         0.         0.         0.         0.        ]
 [0.00080274 0.         0.00065987 0.         0.00083796]
 [0.00082768 0.         0.0005103  0.         0.00454176]
 ...
 [0.01800099 0.         0.00336391 0.         0.26602511]
 [0.         0.16360456 0.         0.13192221 0.20328078]
 [0.         0.         0.         0.05579987 0.31271706]]


## Softmax Activation

### Exponantiation to get rid of negative values

In [18]:
import math

layer_outputs = [4.8, 1.21, 2.385]
E = math.e

exp_values = []
for output in layer_outputs:
    exp_values.append(E**output)

print(exp_values)

[121.51041751873483, 3.353484652549023, 10.859062664920513]


### Normalization

In [19]:
norm_base = sum(exp_values)
norm_values = []

for value in exp_values:
    norm_values.append(value / norm_base)

print(norm_values)
print(sum(norm_values))

[0.8952826639572619, 0.024708306782099374, 0.0800090292606387]
0.9999999999999999


### Using numpy

In [20]:
exp_values = np.exp(layer_outputs)
norm_values = exp_values / np.sum(exp_values)

print(norm_values)

[0.89528266 0.02470831 0.08000903]


### Turning input into batch

In [21]:
layer_outputs = [[4.8, 1.21, 2.385],
                 [8.9, -1.81, 0.2],
                 [1.41, 1.051, 0.026]]

exp_values = np.exp(layer_outputs)
norm_values = exp_values / np.sum(exp_values, axis=1, keepdims=True)

print(norm_values)
print(norm_values[0][0] + norm_values[0][1] + norm_values[0][2])

[[8.95282664e-01 2.47083068e-02 8.00090293e-02]
 [9.99811129e-01 2.23163963e-05 1.66554348e-04]
 [5.13097164e-01 3.58333899e-01 1.28568936e-01]]
0.9999999999999999


In [22]:
class Softmax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilites = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilites

X, y = createData(100, 3)

dense1 = DenseLayer(2, 3)
activation1 = ReLU()
dense2 = DenseLayer(3, 3)
activation2 = Softmax()

dense1.forward(X)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
activation2.forward(dense2.output)

print(activation2.output[:5])

[[0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]]


## Loss function

### Categorical Cross-Entropy

In [23]:
softmax_output = [0.7, 0.1, 0.2]
target_output = [1, 0, 0] # one-hot encoding

loss = -(np.log(softmax_output[0])*target_output[0] + 
         np.log(softmax_output[1])*target_output[1] + 
         np.log(softmax_output[2])*target_output[2])

print(loss)

loss = -(np.log(softmax_output[0]))
print(loss)

0.35667494393873245
0.35667494393873245


In [24]:
softmax_outputs = np.array([[0.7, 0.1, 0.2],
                            [0.1, 0.5, 0.4],
                            [0.02, 0.9, 0.08]])

class_targets = [0, 1, 1] # for example, 0 means cat, 1 means dog, 2 means pigeon, so here the first input is a cat and the two others are dogs

print(softmax_outputs[[0,1,2], class_targets]) # numpy magic, [0,1,2] selects all three rows of inputs
print(softmax_outputs[range(len(softmax_outputs)), class_targets])

neg_loss = -np.log(softmax_outputs[range(len(softmax_outputs)), class_targets])
avg_loss = np.mean(neg_loss)
print(avg_loss)

[0.7 0.5 0.9]
[0.7 0.5 0.9]
0.38506088005216804


**Problem:** neagtive log of 0 is infinity which results in errors

In [25]:
class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss

class CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7) # to resovle our problem

        # check if scalar values or one hot encoded values have been passed
        # scalar: [0, 1]
        # one-hot encoded: [[1,0],[0,1]]
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        else:
            correct_confidences = np.sum(y_pred_clipped*y_true, axis=1)

        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods
    
loss_function = CategoricalCrossentropy()
loss = loss_function.calculate(activation2.output, y)

print("Loss: ", loss)

Loss:  1.0980680308591275


## Optimization

### Random adjustment of weights

In [26]:
X, y = createData(100, 3)

dense1 = DenseLayer(2, 3)
activation1 = ReLU()
dense2 = DenseLayer(3, 3)
activation2 = Softmax()

loss_function = CategoricalCrossentropy()

lowest_loss = 999999
best_dense1_weights = dense1.weights.copy()
best_dense1_biases = dense1.biases.copy()
best_dense2_weights = dense2.weights.copy()
best_dense2_biases = dense2.biases.copy()

for iteration in range(100000):
    dense1.weights += 0.05 * np.random.randn(2, 3)
    dense1.biases += 0.05 * np.random.randn(1, 3)
    dense2.weights += 0.05 * np.random.randn(3, 3)
    dense2.biases += 0.05 * np.random.randn(1, 3)

    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)

    loss = loss_function.calculate(activation2.output, y)

    predictions = np.argmax(activation2.output, axis=1)
    accuracy = np.mean(predictions==y)

    if loss < lowest_loss:
        print("New set of weights found, iteration:", iteration, "loss:", loss, "acc:", accuracy)
        best_dense1_weights = dense1.weights.copy()
        best_dense1_biases = dense1.biases.copy()
        best_dense2_weights = dense2.weights.copy()
        best_dense2_biases = dense2.biases.copy()
        lowest_loss = loss
    else:
        dense1.weights = best_dense1_weights.copy()
        dense1.biases = best_dense1_biases.copy()
        dense2.weights = best_dense2_weights.copy()
        dense2.biases = best_dense2_biases.copy()

New set of weights found, iteration: 0 loss: 1.0995347495967274 acc: 0.3333333333333333
New set of weights found, iteration: 2 loss: 1.098922828015789 acc: 0.3333333333333333
New set of weights found, iteration: 8 loss: 1.0985118206616498 acc: 0.3333333333333333
New set of weights found, iteration: 14 loss: 1.0984489065242378 acc: 0.3333333333333333
New set of weights found, iteration: 17 loss: 1.097436060523383 acc: 0.3433333333333333
New set of weights found, iteration: 22 loss: 1.0971097128231166 acc: 0.4266666666666667
New set of weights found, iteration: 33 loss: 1.0966510923150883 acc: 0.35333333333333333
New set of weights found, iteration: 37 loss: 1.0963794169836545 acc: 0.37
New set of weights found, iteration: 40 loss: 1.0960531146473522 acc: 0.3566666666666667
New set of weights found, iteration: 46 loss: 1.0956507180565485 acc: 0.3566666666666667
New set of weights found, iteration: 47 loss: 1.0934543942627015 acc: 0.39666666666666667
New set of weights found, iteration: 6

New set of weights found, iteration: 4288 loss: 1.0455498145609585 acc: 0.44333333333333336
New set of weights found, iteration: 4774 loss: 1.0455337433010194 acc: 0.43666666666666665
New set of weights found, iteration: 4791 loss: 1.0454022873633046 acc: 0.43666666666666665
New set of weights found, iteration: 4920 loss: 1.0450987837947907 acc: 0.45
New set of weights found, iteration: 5067 loss: 1.0447649000505983 acc: 0.44
New set of weights found, iteration: 5178 loss: 1.0444599840345479 acc: 0.46
New set of weights found, iteration: 5361 loss: 1.0442217068020012 acc: 0.43333333333333335
New set of weights found, iteration: 5372 loss: 1.0441951376214929 acc: 0.43666666666666665
New set of weights found, iteration: 6238 loss: 1.0440896922499492 acc: 0.44333333333333336
New set of weights found, iteration: 7868 loss: 1.0440751638027457 acc: 0.44333333333333336
New set of weights found, iteration: 7959 loss: 1.0439000866669528 acc: 0.44666666666666666
New set of weights found, iterati